In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent
from shell.fleet.data.send_utilize import *

In [3]:
# save_root_dir = "vanilla_results"
save_root_dir = "vanilla_remove_datasets_hack_results"
dataset = "mnist"
algo = "modular"
num_train = 64
seed = 0
use_contrastive = True

In [4]:
job_name = f"{dataset}_{algo}_numtrain_{num_train}"
if use_contrastive:
    job_name += "_contrastive"
experiment = os.path.join(save_root_dir, job_name, dataset,algo, f"seed_{seed}")

In [5]:
config_path = os.path.join(experiment, "hydra_out", ".hydra", "config.yaml")
# read the config file
cfg = omegaconf.OmegaConf.load(config_path)
cfg

{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'no_sharing', 'num_coms_per_round': 0}, 'seed': 0, 'algo': 'modular', 'job_name': 'mnist_modular_numtrain_64_contrastive', 'num_agents': 8, 'root_save_dir': 'vanilla_remove_datasets_hack_results', 'parallel': True, 'num_init_tasks': 4, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': True}}

In [6]:
graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, fleet_additional_cfg = setup_experiment(cfg)
len(datasets)

{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'no_sharing', 'num_coms_per_round': 0}, 'seed': 0, 'algo': 'modular', 'job_name': 'mnist_modular_numtrain_64_contrastive', 'num_agents': 8, 'root_save_dir': 'vanilla_remove_datasets_hack_results', 'parallel': True, 'num_init_tasks': 4, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': True}}
i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init

8

In [7]:
classes_sequence_list = [dataset.class_sequence for dataset in datasets]
classes_sequence_list

[array([5, 0, 4, 3, 4, 6, 4, 3, 8, 0, 1, 0, 5, 9, 6, 4, 9, 0, 3, 0]),
 array([4, 7, 5, 7, 6, 0, 3, 0, 5, 0, 3, 6, 2, 7, 6, 7, 6, 1, 0, 5]),
 array([6, 7, 7, 8, 4, 1, 1, 8, 6, 1, 6, 4, 5, 7, 8, 0, 2, 3, 0, 3]),
 array([2, 8, 0, 3, 7, 4, 3, 4, 4, 5, 9, 3, 0, 6, 9, 1, 3, 1, 7, 9]),
 array([0, 3, 0, 2, 9, 7, 0, 9, 2, 1, 7, 6, 8, 6, 1, 8, 6, 4, 9, 8]),
 array([8, 4, 6, 3, 3, 1, 1, 6, 4, 9, 3, 2, 2, 9, 6, 0, 5, 9, 7, 2]),
 array([7, 5, 8, 4, 6, 9, 8, 3, 4, 6, 1, 3, 3, 1, 4, 1, 9, 2, 6, 2]),
 array([5, 1, 8, 3, 9, 6, 5, 9, 5, 0, 7, 2, 7, 8, 6, 1, 6, 0, 0, 6])]

In [8]:
task_id = 3
num_added_components = None
agent_id = 0

In [9]:
dataset = datasets[agent_id]
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=128,
                                                         shuffle=False,
                                                         num_workers=0,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(dataset.testset[:(task_id+1)])}

In [10]:
agent = Agent(agent_id, seed, dataset,
                NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, 
                cfg.sharing_strategy)

In [11]:
agent.net = load_net(cfg, NetCls, net_cfg, agent_id=agent_id, task_id=task_id, num_added_components=num_added_components)
agent.net

MLPSoftLLDynamic(
  (structure): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (5): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (6): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (7): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (8): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
      (9): Parameter containing: [torch.float32 of size 4x4 (GPU 0)]
  )
  (softmax): Softmax(dim=0)
  (components): ModuleList(
    (0-3): 4 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.0, inplace=False)
  (random_linear_projection): Linear(in_features=784, out_features=64, b

In [12]:
eval_net(agent.net, testloaders)

{0: 0.9668803418803419,
 1: 0.9844377510040161,
 2: 0.9788659793814433,
 3: 0.9799196787148594,
 'avg': 0.9775259377451653}

In [13]:
num_classes_per_task = cfg.dataset.num_classes_per_task
print(set(dataset.class_sequence[:(task_id + 1)* num_classes_per_task]))
print(set(datasets[1].class_sequence[:(task_id + 1)* num_classes_per_task]))

{0, 3, 4, 5, 6}
{0, 3, 4, 5, 6, 7}
